In [55]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca/demod')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
print os.getcwd()
import convert
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
#from pointingtools import compute_parallactic_angle, altaz2ha 

import matplotlib.pyplot as plt
import time
from datetime import timedelta
import numpy.ma as ma

import Tkinter,tkFileDialog

from Tkinter import *
import ttk
import demod

import logging as l

from dtypes import *


%pylab


D:\software_git_repos\Polaris\polaris_software\notebook
Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
def get_filelist(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/polaris/polaris_data/data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
   
        
    return filelist

In [3]:
def plotrawnow(yrmoday,chan,var,fpath,rstep=50,supply_index=False):
	"""
	function to automatically read last science file plot raw data vs encoder
	yrmoday should be a string '20130502' fpath should point to the 
	directory where acq_tel and converter.py were run
	rstep determines how many revolutions to skip between plotted revolutions
	"""
	fld=glob.glob(fpath+'data/'+yrmoday+'/*.dat')
	fld.sort()
	stats=os.stat(fld[-1])
	if stats.st_size == 10752000:
		dr=demod.read_raw([fld[-1]],supply_index=supply_index)
		for i in range(0,np.shape(dr[chan])[0],rstep):
			plt.plot(dr[chan][i,:],label='rev '+str(i))
		plt.xlabel('encoder position')
		plt.ylabel('Signal V')
		plt.title(chan+' Raw data, every '+str(rstep) + ' revs, file: '+fld[-1])
		plt.legend()
		plt.grid()
		plt.show()
	return dr

In [4]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3 ', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'Backend TSS':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Calibrator':'ch15'}

    chan = chans[name]

    return chan

In [57]:
def create_revdata(raw_data, volts=True,supply_index=False, printstatements=True):

    """Deletes invalid revolutions and shapes the array on revolutions
    
    Parameters
    ----------
    raw_data : ndarray
        input array with dtype dat_dtype

    Returns
    -------
    revdata : ndarray
        reshaped output dataset
    """

    # remove partial revolutions at the beginning and end of dataset
    d=raw_data.copy()
    if supply_index==True:
        d['enc']=np.mod(d['enc']+6,4096)
    start_of_revs, = np.where(d['enc'] < config['ENC_START_TRIGGER'])
    if len(start_of_revs)>0:
        d = np.array(d[start_of_revs[0]:start_of_revs[-1]])
    d = demod.remove_noise_triggers(d)

    if printstatements:
        print len(start_of_revs)
        print config['ENC_START_TRIGGER']

    # remove revolutions with bad number of samples 
    start_of_revs, = np.where(d['enc'] < config['ENC_START_TRIGGER'])
    # add the end of array to compute the length of the last revolution
    start_of_revs = np.append(start_of_revs, len(d['enc']))
    samples_per_rev = np.diff(start_of_revs)
    invalid_revs, = np.where(samples_per_rev != config['SEC_PER_REV'])

    if printstatements:
        if len(invalid_revs) > 0:
            l.warning('Removing invalid revolutions (index from beginning of file): %s' % invalid_revs)
        else:
            l.info('No invalid revolutions')

    # remove the samples of the bad revolutions from the array
    #print('dshape',np.shape(d))
    for i in invalid_revs[::-1]:
	d = np.delete(d, np.s_[start_of_revs[i]:start_of_revs[i+1]])
    #print('dshape after cleaning',np.shape(d))

	
    #for i in invalid_revs:
      #  d = np.delete(d, np.s_[start_of_revs[i]:start_of_revs[i+1]])

    out_dtype = rev_dtype if volts else rev_dtype_adu

    # create the rev output array
    if len(d) == 0:
        l.error('NO VALID DATA IN FILE')
        data = np.zeros(0, dtype=out_dtype)
    else:
        data = np.zeros(len(d)/config['SEC_PER_REV'], dtype=out_dtype)
        d_rev = d[::config['SEC_PER_REV']]
	if len(d_rev)>len(data):
		data=np.append(data,data[0])
        data['rev'] = d_rev['rev0'].astype(np.long) + \
                      d_rev['rev1'].astype(np.long) * 256 + \
                      d_rev['rev2'].astype(np.long) * 256**2
        data['azi'] = d_rev['rev0'].astype(np.long)+d_rev['rev1'].astype(np.long)*256
        data['enc'] = d['enc']
        for ch in channels_labels:
            chdata = d[ch].reshape((-1, config['SEC_PER_REV']))
            if volts:
                data[ch] = utils.adu2volts(chdata)
            else:
                data[ch] = chdata 

    return data

In [58]:
def read_raw(filenames, volts=True,supply_index=False, printstatements = True):
    """Reads a list of filenames, creates revdata dataset and concatenates them
    
    Parameters
    ----------
    filenames : list
        list of .dat filenames to read
    volts : bool, optional
        whether to convert to volts or keep ADU

    Returns
    -------
    revdata : array
        reshaped concatenated array
    """
    return np.concatenate(
                [create_revdata(demod.open_raw(f), volts,supply_index=supply_index, printstatements=printstatements) for f in filenames]
                         )


In [132]:
fld = get_filelist()

In [81]:
name  = 'H3HiDC'
chan = rt.nametochan(name)

In [77]:
print (dr[0][0])

57758


In [133]:
rstep = 50
imax = 200#np.shape(dr[chan])[0]
fld.sort()
stats=os.stat(fld[-1])
#if stats.st_size == 10752000:
dr=demod.read_raw([fld[-1]],supply_index=False)
plt.figure()
for i in range(0,imax,rstep):
    plt.plot(dr[chan][i,:],label='rev '+str(i))
plt.xlabel('encoder position')
plt.ylabel('Signal V')
plt.title(name+' Raw data, every '+str(rstep) + ' revs')
plt.legend()
plt.grid()
plt.show()

1001
16


In [109]:
revnum = np.arange(0,np.shape(dr[chan])[0])
plt.figure()
plt.plot(revnum, dr[chan][:,0])
plt.xlabel('rev #')
plt.ylabel('signal (V)')
plt.title('signal for encoder position 0')

In [96]:
plt.figure()
plt.plot(raw_data['enc'])

In [130]:
fld2 = get_filelist()
fld2.sort()

In [131]:
rstep = 5
fld.sort()
fld2.sort()
stats=os.stat(fld[-1])
#if stats.st_size == 10752000:
plt.figure()
dr=demod.read_raw([fld[-1]],supply_index=False)
dr2=demod.read_raw([fld2[-1]],supply_index=False)
#for i in range(0,np.shape(dr[chan])[0],rstep):
for i in range(0,5):
    plt.plot(dr[chan][i*rstep,:],label='gold up, rev '+str(i*rstep))
    plt.plot(dr2[chan][i*rstep,:], '--',label='gold down, rev '+str(i*rstep))
plt.xlabel('encoder position')
plt.ylabel('Signal V')
plt.title(name+' Raw data, every '+str(rstep) + ' revs')
plt.legend()
plt.grid()
plt.show()

1001
16
1000
16
